# Установка зависимостей и библиотек

In [ ]:
!pip uninstall -y unsloth peft
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
import os
import shutil
import zipfile
import json
import torch
import textwrap
from datasets import load_dataset, Dataset
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datetime import datetime

# Проверка на CUDA и GPU

In [ ]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Объявление переменных  и функций

In [ ]:
BORDER_LEN = 150

DATASET_FILENAME = "dataset_full.json"

ARCHIVE_NAME = "conflict_model_export"

MODEL_NAME = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

MAX_SEQ_LENGTH = 2024

SEED = 1025

CLEAR_FOLDER = False

output_dir = "./fine_tuned_model_conflict_export"

In [ ]:
# Отдельная загрузка модели
original_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
    force_download=True
)

In [ ]:
# Загрузка LoRA - low rank adaptation
lora_model = FastLanguageModel.get_peft_model(
    original_model,
    r=256,  # LoRA rank - выше = больше емкость, больше память
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=512,  # LoRA scaling factor (обычно 2x rank)
    lora_dropout=0.1,  # Регуляризация для предотвращения переобучения
    bias="lora_only",  # Обучение смещений только для LoRA слоёв
    use_gradient_checkpointing="unsloth",  # Оптимизированная версия Unsloth
    random_state=SEED,
    use_rslora=True,  # Rank stabilized LoRA для стабильности
    loftq_config=None,  # LoftQ конфигурация
    # init_lora_weights="loftq"  # Инициализация LoftQ для лучшей точности
)

In [ ]:
"""
Иммитация чата.

История чата - массив из след объектов:
{
  "role": "роль"
  "content" : "сообщение"
}

Виды ролей:
system - системный промпт. Указывает модели, кто и что она делает.
user - сообщение отпоьзователя
assistant - сообщение от модели

Стратегия заполнения сообщениями (обязательные правила):
1. Единственный системный промпт - system
2. Пользователь - user
3. Модель - assistant

На каждый user должен быть свой assistant

В конце блока остается одиночное сообщение от user - наш ключивой запрос
"""

MESSAGES = [
    {
        "role": "user",
        "content": "напиши мне конфликт между братом и сестрой"
    },
]

In [ ]:
def generate_response(_model, _tokenizer):
  global MESSAGES

  FastLanguageModel.for_inference(_model)

  # Input like chat
  inputs = _tokenizer.apply_chat_template(
      MESSAGES,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt",
  ).to("cuda")

  # Generate response
  outputs = _model.generate(
      input_ids=inputs,
      max_new_tokens=512,
      use_cache=True,
      temperature=0.7,
      do_sample=True,
      top_p=0.9,
  )

  response = _tokenizer.batch_decode(outputs)[0]

  return response

def format_prompt(example):
  return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'], ensure_ascii=False)}<|endoftext|>"

def format_model_output(response):
  return response.split('<|assistant|>')[-1].split('<|end|>')[0].strip()

def format_text_for_window(text, max_len=BORDER_LEN):
    lines = text.split('\n')
    formatted_lines = []

    for line in lines:
        if len(line) <= max_len:
            formatted_lines.append(line)
        else:
            wrapped = textwrap.fill(line, width=max_len, break_long_words=True, break_on_hyphens=True)
            formatted_lines.extend(wrapped.split('\n'))

    return '\n'.join(formatted_lines)


# Загрузка датасета и адаптация под нужный формат


In [ ]:
file = json.load(open(DATASET_FILENAME, "r"))
formatted_data = [format_prompt(item) for item in file]

DATASET = Dataset.from_dict({"text": formatted_data})

In [ ]:
print("="*BORDER_LEN)
print("ТЕКСТ ЗАПИСИ ИЗ ИСХОДНОГО ДАТАСЕТА:")
print("="*BORDER_LEN)
print(format_text_for_window(json.dumps(file[0], indent=4, ensure_ascii=False)))
print("="*BORDER_LEN)

In [ ]:
print("="*BORDER_LEN)
print("ОТФОРМАТИРОВАННЫЙ ДЛЯ ТРЕНЕРОВКИ ДАТАСЕТ")
print("="*BORDER_LEN)
print(format_text_for_window(formatted_data[0]))
print("="*BORDER_LEN)

# Загрузка модели и тренера из Unsloth

In [ ]:
# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=lora_model,
    tokenizer=tokenizer,
    train_dataset=DATASET,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=20,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=32,  # Effective batch size = 8
        warmup_steps=20,
        num_train_epochs=10,
        learning_rate=1e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=SEED,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)


# Тестирование

При тестировании модели до/после необходимо: провести тест ДО тренеровки, НАТРЕНЕРОВАТЬ модель, ПОСЛЕ получить ответ натренерованной модели

## Тестирование оригинальной модели

In [ ]:
print("="*BORDER_LEN)
print("ОТВЕТ МОДЕЛИ:")
print("="*BORDER_LEN)
print(format_model_output(generate_response(lora_model, tokenizer)))
print("="*BORDER_LEN)

## Тренеровка модели

In [ ]:
trainer_stats = trainer.train()

## Тестирование натренерованной модели

In [ ]:
print("="*BORDER_LEN)
print("ОТВЕТ НАТРЕНЕРОВАННОЙ МОДЕЛИ:")
print("="*BORDER_LEN)
print(format_text_for_window(format_model_output(generate_response(lora_model, tokenizer))))
print("="*BORDER_LEN)

# Статистика

In [ ]:
print("="*BORDER_LEN)
print("ИТОГОВАЯ СТАТИСТИКА ОБУЧЕНИЯ")
print("="*BORDER_LEN)

print(f"Общее количество шагов: {trainer_stats.global_step}")
print(f"Общее время обучения: {trainer_stats.metrics.get('train_runtime', 'N/A')} секунд")
print(f"Средняя скорость обучения: {trainer_stats.metrics.get('train_samples_per_second', 'N/A')} образцов/сек")
print(f"Количество обработанных образцов: {trainer_stats.metrics.get('train_samples', 'N/A')}")

print("\n" + "-"*BORDER_LEN)
print("ПОТЕРИ (LOSS) ПО ЭТАПАМ:")
print("-"*BORDER_LEN)

if hasattr(trainer, 'state') and trainer.state.log_history:
    for i, log_entry in enumerate(trainer.state.log_history[-10:], 1):
        if 'loss' in log_entry:
            step = log_entry.get('step', 'N/A')
            loss = log_entry.get('loss', 'N/A')
            learning_rate = log_entry.get('learning_rate', 'N/A')
            print(f"Шаг {step}: Loss = {loss:.4f}, LR = {learning_rate:.2e}")

print("\n" + "-"*BORDER_LEN)
print("ФИНАЛЬНЫЕ МЕТРИКИ:")
print("-"*BORDER_LEN)

for key, value in trainer_stats.metrics.items():
    print(f"{key}: {value}")

print("\n" + "="*BORDER_LEN)

# Экспорт

In [ ]:
FastLanguageModel.for_inference(lora_model)

os.makedirs(output_dir, exist_ok=True)

try:
    lora_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("="*BORDER_LEN)
    print(f"Файлы модели успешно сохранены в: {output_dir}")
    print("="*BORDER_LEN)
except Exception as e:
    print(f"Ошибка при сохранении модели: {e}")

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"{ARCHIVE_NAME}_{timestamp}.zip"

try:
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(output_dir):
            for file in files:
                file_path = os.path.join(root, file)
                archive_path = os.path.relpath(file_path, output_dir)
                zipf.write(file_path, archive_path)
    print(f"Модель успешно экспортирована в ZIP-архив: {zip_filename}")
    print("="*BORDER_LEN)
except Exception as e:
    print("="*BORDER_LEN)
    print(f"Ошибка при создании ZIP-архива: {e}")
    print("="*BORDER_LEN)

In [ ]:
if CLEAR_FOLDER:
  try:
      shutil.rmtree(output_dir)
      print(f"Временная директория {output_dir} удалена после архивации.")
      print("="*BORDER_LEN)
  except Exception as e:
      print(f"Не удалось удалить временную директорию {output_dir}: {e}")
      print("="*BORDER_LEN)

# Загрузка натренерованной модели

In [ ]:
saved_model_dir = output_dir

try:
    fine_tuned_lora_model, fine_tuned_tokenizer = FastLanguageModel.from_pretrained(
        model_name=saved_model_dir,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype=None,
        load_in_4bit=True,
    )
    print(f"Модель LoRA успешно загружена из: {saved_model_dir}")
except Exception as e:
    print(f"Ошибка при загрузке модели LoRA: {e}")

try:
    fine_tuned_full_model = fine_tuned_lora_model.merge_and_unload()
    print("LoRA адаптация успешно применена. Полная модифицированная модель готова.")
except Exception as e:
    print(f"Ошибка при применении LoRA адаптации: {e}")
    fine_tuned_full_model = fine_tuned_lora_model
    print("Продолжаем с моделью LoRA.")

FastLanguageModel.for_inference(fine_tuned_full_model if fine_tuned_full_model is not None else fine_tuned_lora_model)

In [ ]:
print("="*BORDER_LEN)
print("ОТВЕТ НАТРЕНЕРОВАННОЙ МОДЕЛИ:")
print("="*BORDER_LEN)
print(format_text_for_window(format_model_output(generate_response(fine_tuned_lora_model, fine_tuned_tokenizer))))
print("="*BORDER_LEN)

# Сохранение модели на google disk

In [ ]:
import os
import shutil
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
GOOGLE_DISK_MODEL_FOLDER = "conflict_mistral_model"
LOCAL_MODEL_PATH = output_dir if output_dir is not None else './fine_tuned_model_conflict_export'

In [ ]:
model_folder_path = f'/content/drive/MyDrive/{conflict_mistral_model}'
os.makedirs(model_folder_path, exist_ok=True)

if os.path.exists(LOCAL_MODEL_PATH):
    for item in os.listdir(LOCAL_MODEL_PATH):
        source_item = os.path.join(LOCAL_MODEL_PATH, item)
        dest_item = os.path.join(model_folder_path, item)
        if os.path.isdir(source_item):
            shutil.copytree(source_item, dest_item, dirs_exist_ok=True)
        else:
            shutil.copy2(source_item, dest_item)
    print(f"Модель сохранена {model_folder_path}")
else:
    print(f"Папка с моделью {LOCAL_MODEL_PATH} не найдена.")